In [ ]:
import holoviews as hv
import geoviews as gv
import numpy as np
import pyproj
import xarray as xr
from cartopy import crs
import json
import xarray as xr, hvplot.xarray
from shapely.geometry import Point

hv.extension('bokeh', logo=False)
gv.extension('bokeh', logo=False)

In [ ]:
outcrs = crs.epsg(6676)
url = 'https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{Z}/{X}/{Y}.jpg'
geomap = gv.WMTS(url, crs=outcrs).options(width=600, height=400)

In [ ]:
p1x = np.array( [[ -20500, 74000 ]] )
               
def mkunitvecotr(th):
    ex = np.array( [[1, 0]] )
    th = th * np.pi / 180
    r = np.array( [[np.cos(th), -np.sin(th)],[np.sin(th), np.cos(th)]])
    return np.dot( r, ex.T)

thx = 57
exi = mkunitvecotr(thx)
eeta = mkunitvecotr(thx+90)

Lx, Ly = 6300, 3000 
pathxi = np.c_[p1x.T, p1x.T + exi*Lx]
patheta = np.c_[p1x.T, p1x.T + eeta*Ly]

# drawing using geoviews
l1 = gv.Path(tuple(pathxi), crs=outcrs).options(line_width=2)
l2 = gv.Path(tuple(patheta), crs=outcrs).options(line_width=2)

out = geomap*l1 *l2
# gv.renderer('bokeh').save(out, 'set_axis')
out.options(height=600)

In [ ]:
dx, dy = 10.0, 10.0
mps = np.empty(( int(Lx/dx)+1, int(Ly/dy)+1, 2 ))
for i, _ in enumerate(mps[:,0]):
    for j, _ in enumerate(mps[0]):
        mps[i,j] = (p1x.T + exi*dx*i + eeta*dy*j).flatten()

In [ ]:
import libgsidem2el as gsi
dem = gsi.libgsidem2el('DEM5A')

EPSGout = pyproj.Proj("+init=EPSG:6668")
EPSGin = pyproj.Proj("+init=EPSG:6676")

Z = np.empty_like( mps[:,:,0] ) 
nx, ny = len(Z), len(Z[0])
for j in range(ny) :
    for i in range(nx) :
        p = mps[i, j]
        lon, lat = pyproj.transform( EPSGin, EPSGout, *p)
        val = dem.getEL(lon, lat, zoom = 15) 
#         val = np.nan if val == 'e' else val
        if val == 'e' :
#             print('e')
            val = np.nan
            
#         Z[i,j] = dem.getEL(lon, lat, zoom = 15)
        Z[i,j] = val

In [ ]:
len(mps) *  len(mps[0])

In [ ]:
Qx = mps[:,:,0]
Qy = mps[:,:,1]
ds = xr.Dataset({'elevation': (['x','y'], Z) }, coords={'xc': (('x', 'y'), Qx), 'yc': (('x', 'y'), Qy)})

In [ ]:
out1 = ds.hvplot.quadmesh( 'xc','yc',z='elevation', crs=outcrs, rasterize=True, dynamic=True, cmap='jet',width=600, height=600,colorbar=True ).redim.range(elevation=(320,340))

In [ ]:
point = gv.Points( Point(mps[206,81]).coords[:], crs=outcrs ).options(size=10, color='red') \
      * gv.Points( Point(mps[207,81]).coords[:], crs=outcrs ).options(size=10, color='red') \
      * gv.Points( Point(mps[208,81]).coords[:], crs=outcrs ).options(size=10, color='red') \
      * gv.Points( Point(mps[209,81]).coords[:], crs=outcrs ).options(size=10, color='red') \
      * gv.Points( Point(mps[210,81]).coords[:], crs=outcrs ).options(size=10, color='red')

In [ ]:
g = out1.opts(alpha=0.7) * geomap * point

In [ ]:
g

In [ ]:
hv.save(g,'elevation.html')

In [ ]:
ds.to_netcdf('zb.nc')